### LoRA Fine-tuning Phi1.5

This notebook is made for LoRA fine-tuning Phi1.5. LoRA is a parameter efficient fine-tuning technique that only adjusts few parameters instead of full fine-tuning of the model, thus, it's faster. We will be using [alvarobartt/openhermes-preferences-metamath](https://huggingface.co/datasets/VMware/open-instruct) dataset that has instructions. To apply LoRA, we'll use [PEFT](https://huggingface.co/docs/peft/index) library and for supervised instruction tuning, we will use `SFTTrainer` from [TRL](https://huggingface.co/docs/trl/en/index).

Login to Hugging Face Hub

In [3]:
from huggingface_hub import login

login(
  token="", # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

/home/juancm/miniconda3/envs/gemma/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/juancm/.cache/huggingface/token
Login successful


In [2]:
import wandb
wandb.login(key="")
run = wandb.init(project='Finetuning-microsoft-phi-1.5', job_type="training", anonymous="allow")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jucamohedano. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/juancm/.netrc


In [1]:
import torch
torch.cuda.is_available()

True

Prepare dataset

In [4]:
from datasets import load_dataset

data = load_dataset("alvarobartt/openhermes-preferences-metamath", split="train")

data_template = f"""INPUT_REPLACE_ME

Answer: OUTPUT_REPLACE_ME"""

texts = []
key = "chosen" # only selecting the chosen key
for row in data:
  # for key in row.keys():
  user_prompt = row[key][0]["content"] # user prompt
  response = row[key][1]["content"] # response
  texts.append(data_template.replace("INPUT_REPLACE_ME", user_prompt).replace("OUTPUT_REPLACE_ME", response))
data.add_column("text_column", texts)

if "text_column" in data.column_names:
    data = data.remove_columns("text_column")
data = data.add_column("text_column", texts)

Fine-tune LLM using trl and the SFTTrainer

We'll shrink the model even further by loading it in 4bit using `bitsandbytes`. Then initialize the model with the CausalLM head and initialize the tokenizer.

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig

lora_config = LoraConfig(
    r=6,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj", "fc1", "fc2"],
    # target_modules="all-linear",
    task_type="CAUSAL_LM"
)

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model_id = "microsoft/phi-1_5"
tokenizer = AutoTokenizer.from_pretrained(model_id,)
model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             quantization_config=bnb_config,
                                             attn_implementation="flash_attention_2",
                                             device_map={"":0}) # can also be set to "auto"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right' # to prevent warnings

In [6]:
model

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiFlashAttention2(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (dense): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear4bit(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (final_la

In [7]:
from peft import get_peft_model
get_peft_model(model, lora_config).print_trainable_parameters()

trainable params: 4,718,592 || all params: 1,422,989,312 || trainable%: 0.3315971497613047


Initializing `SFTTrainer` from TRL is all you need!

Small note: if your dataset needs formatting, you can write a formatting function and pass it. You need to either pass `formatting_func` or `dataset_text_field` if your dataset text field doesn't need any formatting and you did your preprocessing beforehand.

Then simply call ` train`. Note that this notebook is built for educational purposes so you might need to adjust the hyperparameters to your own use case.

In [9]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="Phi1.5-openhermes-preferences-metamath", # directory to save and repository id
    num_train_epochs=3,                     # number of training epochs
    per_device_train_batch_size=2,          # batch size per device during training
    gradient_accumulation_steps=16,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    # gradient_checkpointing_kwargs={"use_reentrant": False},
    optim="paged_adamw_8bit",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                    # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="wandb",                # report metrics to tensorboard
)

In [10]:
from trl import SFTTrainer

max_seq_length = 1024 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=data,
    dataset_text_field="text_column",
    peft_config=lora_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    dataset_kwargs={
        "add_special_tokens": False, # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
    }
)

In [11]:
import time
# start training, the model will be automatically saved to the hub and the output directory
start_time = time.time()  # Record the start time
trainer.train()
end_time = time.time()  # Record the end time

training_time = end_time - start_time  # Calculate total training time

print(f"Training completed in {training_time} seconds.")

# save model
trainer.save_model()

  0%|          | 0/1077 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/juancm/miniconda3/envs/gemma/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.
  1%|          | 10/1077 [06:35<10:37:48, 35.87s/it]

{'loss': 1.1367, 'grad_norm': 0.09033203125, 'learning_rate': 0.0002, 'epoch': 0.03}


  2%|▏         | 20/1077 [12:27<10:25:21, 35.50s/it]

{'loss': 1.051, 'grad_norm': 0.09423828125, 'learning_rate': 0.0002, 'epoch': 0.06}


  3%|▎         | 30/1077 [18:19<10:05:32, 34.70s/it]

...

{'loss': 0.713, 'grad_norm': 0.255859375, 'learning_rate': 0.0002, 'epoch': 2.95}


 99%|█████████▉| 1070/1077 [10:32:58<04:11, 35.92s/it]

{'loss': 0.7146, 'grad_norm': 0.265625, 'learning_rate': 0.0002, 'epoch': 2.97}


100%|██████████| 1077/1077 [10:37:16<00:00, 35.50s/it]


{'train_runtime': 38236.6524, 'train_samples_per_second': 0.903, 'train_steps_per_second': 0.028, 'train_loss': 0.7918540915628219, 'epoch': 2.99}
Training completed in 38240.929423093796 seconds.


### Let's do some testing

In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import  AutoTokenizer, pipeline

peft_model_id = "Phi1.5-openhermes-preferences-metamath"

# Load Model with PEFT adapter
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
# get token id for end of conversation
# eos_token = tokenizer("",add_special_tokens=False)["input_ids"][0]

/home/juancm/miniconda3/envs/gemma/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalL

In [7]:


text = "If Bianca worked for 12.5 hours last weekend, Celeste worked twice that amount, and McClain worked 7.5 hours less than Celeste, what is the total number of minutes that the three people worked in total?\n\nAnswer:"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt", return_attention_mask=False).to(device)

outputs = model.generate(**inputs, max_new_tokens=1024)

In [8]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

If Bianca worked for 12.5 hours last weekend, Celeste worked twice that amount, and McClain worked 7.5 hours less than Celeste, what is the total number of minutes that the three people worked in total?

Answer:  First, let's find out how many hours Celeste worked:

Hours worked by Celeste = Hours worked by Bianca * 2
                          = 12.5 hours * 2
                          = 25 hours

Now, let's find out how many hours McClain worked:

Hours worked by McClain = Hours worked by Celeste - 7.5 hours
                        = 25 hours - 7.5 hours
                        = 17.5 hours

Next, we need to convert the hours to minutes:

Minutes worked by Celeste = Hours worked by Celeste * 60 minutes/hour
                            = 25 hours * 60 minutes/hour
                            = 1500 minutes

Minutes worked by McClain = Hours worked by McClain * 60 minutes/hour
                          = 17.5 hours * 60 minutes/hour
                          = 1050 minutes

Finally, we 

In [25]:
import gc
import torch

torch.cuda.empty_cache()


del tokenizer
del model
del trainer

collected = gc.collect()
print(collected)

10150
